# SOLUCIONES - SESIÓN 2: Inyección de Dependencias y Arquitectura Modular

**Contenido:** Soluciones completas y comentadas de los 10 ejercicios.

**Nota:** Todas las soluciones usan `Annotated` para definir tipos reutilizables.

## CONFIGURACIÓN DEL ENTORNO

In [ ]:
# Verificación de Python
import sys
print(f"Python: {sys.version}")
assert sys.version_info >= (3, 8), "Se requiere Python 3.8 o superior"

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 -q
print("Dependencias instaladas")

In [ ]:
# Importaciones globales
from fastapi import FastAPI, Depends, HTTPException, Header, APIRouter, status, Query
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field
from typing import Optional, Annotated, Generator, List
import time

print("Imports completados")

---

## SOLUCIÓN 1: Dependencia Simple que Lee Header

In [ ]:
# Dependencia que lee el header accept-language
def obtener_idioma(accept_language: str = Header(default="es")) -> str:
    """Extrae el idioma del header HTTP."""
    return accept_language

# Tipo reutilizable usando Annotated
IdiomaDep = Annotated[str, Depends(obtener_idioma)]

app = FastAPI()

@app.get("/saludo")
def saludar(idioma: IdiomaDep):
    """Devuelve un saludo en el idioma especificado."""
    # Diccionario de saludos
    saludos = {
        "es": "Hola",
        "en": "Hello",
        "fr": "Bonjour"
    }
    
    # Obtiene el saludo o usa español por defecto
    mensaje = saludos.get(idioma, "Hola")
    
    return {"mensaje": mensaje}

# Pruebas
client = TestClient(app)

# Prueba 1: Idioma por defecto (español)
r1 = client.get("/saludo")
assert r1.status_code == 200
assert r1.json()["mensaje"] == "Hola", f"Error: {r1.json()}"
print("✅ Prueba 1 pasada: Saludo en español")

# Prueba 2: Idioma inglés
r2 = client.get("/saludo", headers={"accept-language": "en"})
assert r2.json()["mensaje"] == "Hello"
print("✅ Prueba 2 pasada: Saludo en inglés")

---

## SOLUCIÓN 2: Dependencia con Validación

In [ ]:
# Dependencia que valida la API key
def validar_api_key(x_api_key: str = Header()) -> str:
    """Valida que la API key sea correcta."""
    API_KEY_VALIDA = "clave-secreta-123"
    
    if x_api_key != API_KEY_VALIDA:
        raise HTTPException(
            status_code=403,
            detail="API key inválida"
        )
    
    return x_api_key

# Tipo reutilizable
ApiKeyValidada = Annotated[str, Depends(validar_api_key)]

app = FastAPI()

@app.get("/datos-privados")
def obtener_datos_privados(api_key: ApiKeyValidada):
    """Devuelve datos confidenciales (requiere API key válida)."""
    return {
        "datos": "información confidencial",
        "api_key": api_key
    }

# Pruebas
client = TestClient(app)

# Prueba 1: API key válida
r1 = client.get("/datos-privados", headers={"x-api-key": "clave-secreta-123"})
assert r1.status_code == 200
assert "información confidencial" in r1.json()["datos"]
print("✅ Prueba 1 pasada: API key válida")

# Prueba 2: API key inválida (debe dar 403)
r2 = client.get("/datos-privados", headers={"x-api-key": "incorrecta"})
assert r2.status_code == 403, f"Debería ser 403, pero fue {r2.status_code}"
print("✅ Prueba 2 pasada: API key inválida rechazada")

---

## SOLUCIÓN 3: Clase Servicio Inyectable

In [ ]:
# Servicio con lógica de negocio
class CalculadoraService:
    """Servicio para operaciones matemáticas."""
    
    def sumar(self, a: int, b: int) -> int:
        """Suma dos números."""
        return a + b
    
    def multiplicar(self, a: int, b: int) -> int:
        """Multiplica dos números."""
        return a * b

# Dependencia que crea el servicio
def obtener_calculadora() -> CalculadoraService:
    """Factory function para el servicio."""
    return CalculadoraService()

# Tipo reutilizable
CalculadoraDep = Annotated[CalculadoraService, Depends(obtener_calculadora)]

app = FastAPI()

@app.get("/sumar")
def sumar_numeros(a: int, b: int, calculadora: CalculadoraDep):
    """Endpoint para sumar dos números."""
    resultado = calculadora.sumar(a, b)
    return {"resultado": resultado}

@app.get("/multiplicar")
def multiplicar_numeros(a: int, b: int, calculadora: CalculadoraDep):
    """Endpoint para multiplicar dos números."""
    resultado = calculadora.multiplicar(a, b)
    return {"resultado": resultado}

# Pruebas
client = TestClient(app)

# Prueba sumar
r1 = client.get("/sumar?a=5&b=3")
assert r1.status_code == 200
assert r1.json()["resultado"] == 8
print("✅ Suma correcta: 5 + 3 = 8")

# Prueba multiplicar
r2 = client.get("/multiplicar?a=4&b=2")
assert r2.json()["resultado"] == 8
print("✅ Multiplicación correcta: 4 * 2 = 8")

---

## SOLUCIÓN 4: Dependencia con Yield (Simular BD)

In [ ]:
# Clase que simula una caché
class FakeCache:
    """Simula un sistema de caché con conexión."""
    
    def __init__(self):
        self.connected = False
        self.data = {}
    
    def connect(self):
        """Establece conexión con la caché."""
        print("🔌 Conectando cache...")
        self.connected = True
    
    def disconnect(self):
        """Cierra la conexión con la caché."""
        print("🔌 Desconectando cache...")
        self.connected = False
    
    def get(self, key: str):
        """Obtiene un valor de la caché."""
        return self.data.get(key)
    
    def set(self, key: str, value: str):
        """Guarda un valor en la caché."""
        self.data[key] = value

# Dependencia con yield (setup y teardown)
def obtener_cache() -> Generator[FakeCache, None, None]:
    """Gestiona el ciclo de vida del cache."""
    cache = FakeCache()
    
    # Setup: se ejecuta antes del endpoint
    cache.connect()
    
    try:
        # Yield: entrega el cache al endpoint
        yield cache
    finally:
        # Teardown: se ejecuta siempre al finalizar
        cache.disconnect()

# Tipo reutilizable
CacheDep = Annotated[FakeCache, Depends(obtener_cache)]

app = FastAPI()

@app.post("/cache/set")
def guardar_en_cache(key: str, value: str, cache: CacheDep):
    """Guarda un valor en el cache."""
    cache.set(key, value)
    return {"mensaje": "guardado"}

@app.get("/cache/get")
def obtener_de_cache(key: str, cache: CacheDep):
    """Obtiene un valor del cache."""
    valor = cache.get(key)
    return {"valor": valor}

# Pruebas
client = TestClient(app)

# Guardar valor en cache
r1 = client.post("/cache/set?key=nombre&value=Ana")
assert r1.status_code == 200
assert r1.json()["mensaje"] == "guardado"
print("✅ Valor guardado en cache")

# Recuperar valor (⚠️ NOTA: El cache se reinicia en cada request, ¡esperar None!)
r2 = client.get("/cache/get?key=nombre")
assert r2.status_code == 200
print(f"⚠️ Valor recuperado: {r2.json()} (puede ser None por nueva instancia)")

---

## SOLUCIÓN 5: Sub-dependencia

In [ ]:
# Dependencia 1: Obtiene timestamp actual
def obtener_timestamp() -> float:
    """Devuelve el timestamp actual."""
    return time.time()

# Tipo para la primera dependencia
TimestampDep = Annotated[float, Depends(obtener_timestamp)]

# Dependencia 2: Usa el timestamp de la dependencia 1
def obtener_logger(timestamp: TimestampDep) -> dict:
    """Crea un logger con el timestamp inyectado."""
    return {
        "timestamp": timestamp,
        "nivel": "INFO"
    }

# Tipo para la segunda dependencia
LoggerDep = Annotated[dict, Depends(obtener_logger)]

app = FastAPI()

@app.get("/log")
def obtener_log(logger: LoggerDep):
    """Devuelve información del log."""
    return {
        "log": logger,
        "mensaje": "Registro creado"
    }

# Pruebas
client = TestClient(app)

r = client.get("/log")
assert r.status_code == 200
assert "log" in r.json()
assert "timestamp" in r.json()["log"]
assert r.json()["log"]["nivel"] == "INFO"
print("✅ Log generado correctamente:", r.json())

---

## SOLUCIÓN 6: APIRouter Básico

In [ ]:
# Crear el router sin configuración
router = APIRouter()

@router.get("/items")
def listar_items():
    """Lista todos los items."""
    return {"items": ["item1", "item2", "item3"]}

@router.get("/items/{item_id}")
def obtener_item(item_id: int):
    """Obtiene un item específico por ID."""
    return {
        "item_id": item_id,
        "nombre": f"Item {item_id}"
    }

# Crear app y registrar router
app = FastAPI()
app.include_router(router)

# Pruebas
client = TestClient(app)

r1 = client.get("/items")
assert r1.status_code == 200
assert "items" in r1.json()
print("✅ Listar items:", r1.json())

r2 = client.get("/items/5")
assert r2.json()["item_id"] == 5
print("✅ Obtener item específico:", r2.json())

---

## SOLUCIÓN 7: Router con Prefijo y Tags

In [ ]:
# Router con prefijo y tags
router_productos = APIRouter(
    prefix="/api/v1/productos",
    tags=["Productos"]
)

@router_productos.get("/")
def listar_productos():
    """Lista todos los productos."""
    return {
        "productos": [
            {"id": 1, "nombre": "Laptop"},
            {"id": 2, "nombre": "Mouse"}
        ]
    }

@router_productos.post("/")
def crear_producto(nombre: str):
    """Crea un nuevo producto."""
    return {
        "id": 3,
        "nombre": nombre,
        "mensaje": "creado"
    }

@router_productos.delete("/{producto_id}")
def eliminar_producto(producto_id: int):
    """Elimina un producto por ID."""
    return {
        "mensaje": f"Producto {producto_id} eliminado"
    }

# Crear app y registrar router
app = FastAPI()
app.include_router(router_productos)

# Pruebas
client = TestClient(app)

r1 = client.get("/api/v1/productos")
assert r1.status_code == 200
print("✅ Listar productos:", r1.json())

r2 = client.post("/api/v1/productos?nombre=Laptop")
assert r2.json()["nombre"] == "Laptop"
print("✅ Producto creado:", r2.json())

r3 = client.delete("/api/v1/productos/1")
assert "eliminado" in r3.json()["mensaje"]
print("✅ Producto eliminado:", r3.json())

---

## SOLUCIÓN 8: Dependencias Globales en Router

In [ ]:
# Dependencia que verifica rol de administrador
def verificar_rol_admin(x_role: str = Header()) -> bool:
    """Valida que el usuario tenga rol admin."""
    if x_role != "admin":
        raise HTTPException(
            status_code=403,
            detail="Acceso denegado: se requiere rol admin"
        )
    return True

# Router con dependencia global (se aplica a TODOS los endpoints)
router_admin = APIRouter(
    prefix="/admin",
    tags=["Admin"],
    dependencies=[Depends(verificar_rol_admin)]  # <-- Dependencia global
)

@router_admin.get("/usuarios")
def admin_usuarios():
    """Devuelve lista de usuarios (solo admin)."""
    return {"usuarios": ["todos los usuarios del sistema"]}

@router_admin.get("/configuracion")
def admin_configuracion():
    """Devuelve configuración del sistema (solo admin)."""
    return {"config": {"modo": "producción", "debug": False}}

# Crear app y registrar router
app = FastAPI()
app.include_router(router_admin)

# Pruebas
client = TestClient(app)

# Con rol admin (debe funcionar)
r1 = client.get("/admin/usuarios", headers={"x-role": "admin"})
assert r1.status_code == 200
print("✅ Acceso admin a /usuarios:", r1.json())

# Con rol user (debe dar 403)
r2 = client.get("/admin/usuarios", headers={"x-role": "user"})
assert r2.status_code == 403, f"Debería ser 403, pero fue {r2.status_code}"
print("✅ Acceso denegado correctamente")

# Endpoint configuración con admin
r3 = client.get("/admin/configuracion", headers={"x-role": "admin"})
assert "config" in r3.json()
print("✅ Acceso admin a /configuracion:", r3.json())

---

## SOLUCIÓN 9: Servicio Completo con Múltiples Métodos

In [ ]:
# ⚠️ IMPORTANTE: Simulación de persistencia
# Variable global que simula una BD externa
TAREAS_DB = []  # Simula base de datos externa
NEXT_ID = {"valor": 1}  # Contador global (dict para mutabilidad)

# Servicio para gestión de tareas
class TareaService:
    """Servicio CRUD para gestión de tareas."""
    
    def __init__(self):
        # Usamos la BD global, no creamos una nueva lista
        self.tareas = TAREAS_DB
        self.next_id = NEXT_ID
    
    def crear(self, titulo: str) -> dict:
        """Crea una nueva tarea."""
        tarea = {
            "id": self.next_id["valor"],
            "titulo": titulo,
            "completada": False
        }
        self.tareas.append(tarea)
        self.next_id["valor"] += 1
        return tarea
    
    def listar(self) -> List[dict]:
        """Lista todas las tareas."""
        return self.tareas
    
    def completar(self, tarea_id: int) -> dict:
        """Marca una tarea como completada."""
        for tarea in self.tareas:
            if tarea["id"] == tarea_id:
                tarea["completada"] = True
                return tarea
        
        # Si no existe, lanzar error
        raise HTTPException(
            status_code=404,
            detail="Tarea no encontrada"
        )
    
    def eliminar(self, tarea_id: int) -> dict:
        """Elimina una tarea por ID."""
        for i, tarea in enumerate(self.tareas):
            if tarea["id"] == tarea_id:
                self.tareas.pop(i)
                return {"mensaje": "Tarea eliminada"}
        
        # Si no existe, lanzar error
        raise HTTPException(
            status_code=404,
            detail="Tarea no encontrada"
        )

# Dependencia y tipo
def obtener_tarea_service() -> TareaService:
    return TareaService()

TareaServiceDep = Annotated[TareaService, Depends(obtener_tarea_service)]

app = FastAPI()

@app.post("/tareas")
def crear_tarea(titulo: str, service: TareaServiceDep):
    """Crea una nueva tarea."""
    return service.crear(titulo)

@app.get("/tareas")
def listar_tareas(service: TareaServiceDep):
    """Lista todas las tareas."""
    return service.listar()

@app.put("/tareas/{tarea_id}/completar")
def completar_tarea(tarea_id: int, service: TareaServiceDep):
    """Marca una tarea como completada."""
    return service.completar(tarea_id)

@app.delete("/tareas/{tarea_id}")
def eliminar_tarea(tarea_id: int, service: TareaServiceDep):
    """Elimina una tarea."""
    return service.eliminar(tarea_id)

# Pruebas
TAREAS_DB.clear()  # Limpia BD antes de probar
NEXT_ID["valor"] = 1
client = TestClient(app)

r1 = client.post("/tareas?titulo=Comprar pan")
assert r1.status_code == 200, f"Error POST: {r1.status_code}"
print("✅ Tarea creada:", r1.json())

r2 = client.get("/tareas")
assert len(r2.json()) == 1, "La tarea no se guardó"
print("✅ Tareas listadas:", r2.json())

r3 = client.put("/tareas/1/completar")
assert r3.json()["completada"] == True, "No se marcó como completada"
print("✅ Tarea completada:", r3.json())

r4 = client.delete("/tareas/1")
assert r4.status_code == 200, "Error al eliminar"
print("✅ Tarea eliminada:", r4.json())

---

## SOLUCIÓN 10: Arquitectura Completa (Integrador)

In [ ]:
# ⚠️ IMPORTANTE: Persistencia en memoria con variable global
DB_PRODUCTOS = [
    {"id": 1, "nombre": "Laptop", "precio": 1000},
    {"id": 2, "nombre": "Mouse", "precio": 25}
]

# Clase que simula una base de datos
class FakeDatabase:
    """Simula una BD con conexión y consultas."""
    
    def __init__(self):
        self.connected = False
        self.productos = DB_PRODUCTOS  # Referencia a la BD global
    
    def connect(self):
        """Establece conexión."""
        print("📦 Conectando BD...")
        self.connected = True
    
    def disconnect(self):
        """Cierra conexión."""
        print("📦 Desconectando BD...")
        self.connected = False
    
    def query(self, sql: str):
        """Ejecuta una consulta (simplificado)."""
        return self.productos

# Dependencia con yield para gestionar BD
def obtener_db() -> Generator[FakeDatabase, None, None]:
    """Gestiona el ciclo de vida de la BD."""
    db = FakeDatabase()
    db.connect()
    try:
        yield db
    finally:
        db.disconnect()

DatabaseDep = Annotated[FakeDatabase, Depends(obtener_db)]

# Servicio de productos que usa la BD
class ProductoService:
    """Servicio de negocio para productos."""
    
    def __init__(self, db: FakeDatabase):
        self.db = db
    
    def listar(self) -> List[dict]:
        """Lista todos los productos."""
        return self.db.query("SELECT * FROM productos")
    
    def crear(self, nombre: str, precio: float) -> dict:
        """Crea un nuevo producto."""
        # Calcula nuevo ID
        if self.db.productos:
            nuevo_id = max(p["id"] for p in self.db.productos) + 1
        else:
            nuevo_id = 1
        
        nuevo_producto = {
            "id": nuevo_id,
            "nombre": nombre,
            "precio": precio
        }
        
        self.db.productos.append(nuevo_producto)
        return nuevo_producto

# Dependencia del servicio (inyecta la BD)
def obtener_producto_service(db: DatabaseDep) -> ProductoService:
    """Factory para ProductoService."""
    return ProductoService(db)

ProductoServiceDep = Annotated[ProductoService, Depends(obtener_producto_service)]

# Router organizado
router_productos = APIRouter(
    prefix="/api/productos",
    tags=["Productos"]
)

@router_productos.get("/")
def listar_productos(service: ProductoServiceDep):
    """Lista todos los productos."""
    return service.listar()

@router_productos.post("/")
def crear_producto(nombre: str, precio: float, service: ProductoServiceDep):
    """Crea un nuevo producto."""
    return service.crear(nombre, precio)

# Crear app y registrar router
app = FastAPI()
app.include_router(router_productos)

# Pruebas
client = TestClient(app)

# Listar productos iniciales
r1 = client.get("/api/productos")
assert r1.status_code == 200
assert len(r1.json()) == 2, "Deberían existir 2 productos iniciales"
print("✅ Productos iniciales:", r1.json())

# Crear nuevo producto
r2 = client.post("/api/productos?nombre=Teclado&precio=75")
assert r2.status_code == 200
assert r2.json()["nombre"] == "Teclado"
print("✅ Producto creado:", r2.json())

# Verificar que se guardó (debe haber 3 ahora)
r3 = client.get("/api/productos")
assert len(r3.json()) == 3, "Ahora deberían ser 3 productos"
print("✅ Producto persistió correctamente:", r3.json())